# Web Scraping em Python – Parte Pré-BeautifulSoup

Este notebook é **quase só texto em Markdown** para servir como material didático.

O foco aqui é **tudo o que acontece *antes* do BeautifulSoup**:

- O que é uma **requisição** (request)
- O que é um **protocolo** (especialmente o HTTP)
- O que é uma **resposta** (response)
- Onde entram **headers**, **status codes** (200, 404, 500…), etc.

Não vamos entrar em detalhes de parsing com BeautifulSoup – a ideia é que você entenda o "**mundo da requisição**" primeiro.

## 1. Visão geral da conversa com um site

Quando você acessa um site – seja pelo navegador ou pelo Python – acontece uma **conversa** entre o seu computador e o servidor do site.

Podemos imaginar isso como a troca de **cartas**:

1. Você manda uma carta para o site pedindo alguma coisa (por exemplo, a página inicial).
2. O site recebe essa carta, entende o pedido e responde com outra carta, contendo o que você pediu (por exemplo, o HTML da página).

Nessa analogia:

- A **carta que você manda** é a **requisição** (*request*).
- A **carta que o site responde** é a **resposta** (*response*).
- As **regras de como as cartas devem ser escritas e lidas** são o **protocolo HTTP**.

Web scraping nada mais é do que **automatizar** esse processo: em vez de você abrir o navegador manualmente, o seu código Python manda essa carta, recebe a resposta e depois analisa o conteúdo (por exemplo, com BeautifulSoup).

## 2. O que é uma requisição (request)?

Vamos começar como se fosse para uma pessoa totalmente leiga.

### 2.1. A ideia geral

**Fazer uma requisição** é simplesmente **fazer um pedido para um site**, dizendo basicamente:

> "Oi, site X, por favor me envie tal página / tal informação."

Esse pedido é enviado seguindo um conjunto de regras. Essas regras definem **que informações precisam estar presentes** e **em que formato**. É aí que entra o protocolo HTTP.

### 2.2. O que vai dentro de uma requisição

Uma requisição HTTP (a carta que você manda) tem, em geral, quatro partes importantes:

1. **Método HTTP** (por exemplo, `GET`, `POST`)
2. **URL** (o endereço da coisa que você quer)
3. **Headers** (cabeçalhos – informações extras sobre o pedido)
4. **Corpo (body)** – opcional (dados que você envia, principalmente em `POST`/`PUT`)

Vamos ver cada um com uma analogia simples.

### 2.3. Método HTTP – o "tipo" de pedido

O **método HTTP** indica **o tipo de operação** que você quer fazer. Em analogia com uma carta:

- `GET` → "Só quero **pegar** uma informação" (ex.: me envie a página inicial)
- `POST` → "Quero **enviar** dados" (ex.: enviar um formulário de login)
- `PUT` → "Quero **atualizar** alguma coisa" (ex.: editar um recurso)
- `DELETE` → "Quero **apagar** alguma coisa"

No dia a dia do web scraping, o mais comum de longe é o **`GET`** – que é o método usado para **pedir páginas**.

Então, quando o seu código faz algo como `requests.get("https://exemplo.com")`, por trás dos panos está acontecendo um **GET** para aquela URL.

### 2.4. URL – o endereço do recurso

A **URL** é o endereço daquilo que você quer acessar. Exemplo:

```text
https://www.exemplo.com/produtos?page=2
```

Podemos decompor assim:

- `https://` → indica o protocolo usado (HTTPS, que é HTTP com segurança/criptografia)
- `www.exemplo.com` → domínio do site (como o nome da loja)
- `/produtos` → caminho (qual "sessão" da loja você quer)
- `?page=2` → parâmetros (por exemplo, "quero a página 2 da lista de produtos")

Quando você faz uma requisição, você sempre manda uma URL – é a forma de dizer **aonde** a carta deve chegar e **o que** você está pedindo.

### 2.5. Headers – informações extras sobre o pedido

Os **headers** (cabeçalhos) são como **informações extras coladas na carta** que você envia.

Eles servem para o servidor saber mais detalhes sobre **quem está pedindo** e **como deve responder**.

Alguns exemplos importantes:

- `User-Agent` → diz qual "programa" está fazendo o pedido (Chrome, Firefox, Python etc.)
- `Accept-Language` → indica o idioma preferido (por exemplo, `pt-BR`)
- `Cookie` → carrega dados da sessão (por exemplo, se você está logado)
- `Referer` → indica de qual página você veio

Em scraping, o header mais famoso é o `User-Agent`, porque muitos sites bloqueiam requisições que parecem ser de scripts (como `python-requests`) e só permitem o acesso se o `User-Agent` parecer um navegador de verdade.

### 2.6. Corpo da requisição (body)

O **corpo** (body) é a parte da requisição onde você envia **dados**. Ele aparece principalmente em métodos como `POST` e `PUT`.

Exemplos:

- Enviar um formulário de login (usuário e senha)
- Enviar dados de um cadastro
- Enviar um JSON para uma API

Para um simples `GET` de página (o uso mais básico em scraping), normalmente **não há corpo**: você só manda o método, a URL e os headers.

## 3. O que é um protocolo (especialmente o HTTP)?

Agora vamos falar da palavra que assusta: **protocolo**.

### 3.1. Protocolo de forma bem leiga

Pense em **protocolo** como um conjunto de **regras combinadas** que dizem como duas partes devem se comunicar.

Na vida real:

- Protocolo de atendimento em um hospital
- Protocolo diplomático entre países

Na internet, o protocolo HTTP é o conjunto de regras que define **como a carta (requisição) deve ser escrita** e **como a resposta deve ser enviada**.

Então:

- **Protocolo HTTP** (HyperText Transfer Protocol) → regras para trocas de mensagens entre cliente (seu computador) e servidor (site).
- **HTTPS** = HTTP + camada de segurança (criptografia), como se a mensagem fosse enviada **dentro de um cofre** que só o servidor consegue abrir.

Sem um protocolo comum, cada site e cada navegador teriam seu jeito próprio de comunicar – seria um caos. O HTTP é o idioma padrão que todos seguem.

### 3.2. Como o protocolo organiza a requisição

O protocolo HTTP define:

1. Que a primeira linha da requisição deve ter o **método**, o **caminho** e a **versão do protocolo**.
2. Que logo abaixo vêm os **headers**, um por linha.
3. Que depois de uma linha em branco, pode vir o **corpo** (se houver).

Um exemplo **simplificado** de requisição HTTP crua (só para ilustrar a estrutura) seria:

```http
GET /produtos?page=2 HTTP/1.1
Host: www.exemplo.com
User-Agent: Mozilla/5.0
Accept-Language: pt-BR,pt;q=0.9,en;q=0.8

(corpo – geralmente vazio em GET)
```

Você não precisa escrever isso na mão: o `requests` faz tudo isso para você. Mas entender o formato ajuda a ter clareza sobre o que é uma **requisição**.

## 4. O que é uma resposta (response)?

Se a requisição é a carta que você envia, a **resposta** é a carta que você recebe de volta do servidor.

Essa resposta também segue o protocolo HTTP e contém três partes principais:

1. Uma **linha de status** (com o famoso número 200, 404, 500…)
2. **Headers da resposta** (informações sobre o que está sendo enviado)
3. O **corpo da resposta** (o conteúdo em si: HTML, JSON, imagem etc.)

### 4.1. Linha de status

A linha de status diz se deu tudo certo ou não. Ela tem um **código numérico** (status code) e uma frase.

Exemplos:

- `200 OK` → deu tudo certo, aqui está o conteúdo
- `404 Not Found` → não encontrei o que você pediu
- `500 Internal Server Error` → problema no servidor

### 4.2. Headers da resposta

Assim como na requisição, a resposta também tem **headers**. Alguns exemplos:

- `Content-Type` → tipo de conteúdo (`text/html`, `application/json`, `image/png`...)
- `Content-Length` → tamanho do conteúdo
- `Set-Cookie` → o servidor está mandando cookies para o cliente

### 4.3. Corpo da resposta (body)

O **corpo** é o conteúdo principal.

- Se você pediu uma página web, o corpo será o **HTML** dessa página.
- Se você chamou uma API, pode ser um **JSON**.
- Se é uma imagem, serão os **bytes** da imagem.

Em web scraping com BeautifulSoup, é esse **HTML do corpo da resposta** que você passa para o parser.

## 5. Status codes: entendendo 200, 404, 500 (sem frescura)

Os **status codes** são números que indicam a situação da resposta.

Os principais grupos são:

- **2xx – Sucesso**
  - `200 OK` → deu certo, conteúdo entregue.
- **3xx – Redirecionamento**
  - Indica que o recurso foi movido e o cliente deve ir para outro lugar.
- **4xx – Erro do cliente** (o pedido tem algo de errado do ponto de vista do servidor)
  - `404 Not Found` → o que você pediu não existe.
  - `403 Forbidden` → você não tem permissão.
  - `401 Unauthorized` → você precisa estar autenticado (logado).
- **5xx – Erro do servidor** (problema do lado do site)
  - `500 Internal Server Error` → erro genérico no servidor.
  - `503 Service Unavailable` → serviço indisponível ou sobrecarregado.

Esses códigos são muito úteis em scraping para você saber **se vale a pena tentar parsear o HTML** ou se houve algum problema antes disso.

## 6. Como isso aparece no Python (requests) – só para visualizar

Embora o foco aqui seja o conceito, vale ver rapidamente **como esses elementos aparecem na prática** com a biblioteca `requests` (que é o jeito mais comum de fazer requisições em Python):

```python
import requests

url = "https://example.com"
response = requests.get(url)

# Código de status (200, 404, 500...)
print(response.status_code)

# Headers da resposta
print(response.headers)

# Corpo da resposta como texto (para HTML)
html = response.text
print(html[:500])  # mostra só o começo
```

Repare que **tudo isso acontece antes de tocar no BeautifulSoup**. O BeautifulSoup entra apenas depois, quando você já tem o HTML em mãos e quer **navegar** nele.

Por exemplo:

```python
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
titulo = soup.find("title").get_text(strip=True)
print(titulo)
```

A parte "mágica" de scraping começa a ficar muito mais clara quando você entende essa etapa prévia de **requisição → protocolo → resposta**.

## 7. Resumão da parte pré-BeautifulSoup

- **Requisição (request)**: é o pedido que você manda para o site.
  - Tem método (`GET`, `POST`...), URL, headers e, às vezes, um corpo.
- **Protocolo HTTP**: são as regras que definem **como** essa requisição e essa resposta são escritas e lidas.
- **Resposta (response)**: é a carta que volta do servidor.
  - Tem um status code (200, 404, 500...), headers e um corpo (HTML, JSON, etc.).
- Só depois de tudo isso é que entra o **BeautifulSoup**, que recebe o HTML e faz o trabalho de parser (análise e navegação da estrutura).

Se você entende bem essa parte pré-BeautifulSoup, fica muito mais fácil:

- Diagnosticar erros (por exemplo, por que está vindo 403 em vez de 200?).
- Simular um navegador ajustando headers corretamente.
- Saber quando o problema é de **requisição/protocolo** e quando é de **parser/HTML**.

Na prática: **web scraping começa com boas requisições**. O parser é o próximo capítulo.